In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from tqdm.notebook import tqdm  # Use tqdm.notebook for Jupyter Notebook

In [4]:
# Parameters
total_samples = 10 # low for testing, should be on 20k
batch_size = 4
num_epochs = 10
learning_rate = 1e-3
commitment_cost = 0.25
hidden_channels = 128
embedding_dim = 64
num_embeddings = 512

# Directories containing the images
dataset_dirs = [
    '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ma-boston/parcels',
    '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/nc-charlotte/parcels', 
    '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ny-manhattan/parcels', 
    '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/pa-pittsburgh/parcels'  
]

# Output directory for saving the trained model
output_dir = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Auto-Encoder/Trained Model'  # Replace with the actual path to your output directory

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

In [5]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((16, 16)),  # Resize to a fixed size
    transforms.ToTensor(),        # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# Collect all image paths
all_image_paths = []
for dataset_dir in dataset_dirs:
    for root, _, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):  # Consider image files only
                all_image_paths.append(os.path.join(root, file))

# Randomly sample the images from the collected paths
sampled_image_paths = random.sample(all_image_paths, total_samples)

# Custom dataset to load images from the sampled paths
class SampledImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB format
        if self.transform:
            image = self.transform(image)
        return image, 0  # Returning 0 as a placeholder label

# Create a dataset and dataloader for the sampled images
sampled_dataset = SampledImageDataset(sampled_image_paths, transform=transform)
dataloader = DataLoader(sampled_dataset, batch_size=batch_size, shuffle=True)

# VQ-VAE Model Definition
class Encoder(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_embeddings, embedding_dim):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(hidden_channels, embedding_dim, kernel_size=4, stride=2, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.conv2(x)
        return x

class Decoder(nn.Module):
    def __init__(self, embedding_dim, hidden_channels, out_channels):
        super(Decoder, self).__init__()
        self.conv1 = nn.ConvTranspose2d(embedding_dim, hidden_channels, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose2d(hidden_channels, out_channels, kernel_size=4, stride=2, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = torch.tanh(self.conv2(x))
        return x

class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost):
        super(VectorQuantizer, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.uniform_(-1/self.num_embeddings, 1/self.num_embeddings)
        self.commitment_cost = commitment_cost

    def forward(self, x):
        # Quantize
        flattened = x.view(-1, self.embedding_dim)
        distances = torch.cdist(flattened, self.embedding.weight)
        encoding_indices = torch.argmin(distances, dim=1)
        quantized = self.embedding(encoding_indices).view(x.size())

        # Loss
        e_latent_loss = F.mse_loss(quantized.detach(), x)
        q_latent_loss = F.mse_loss(quantized, x.detach())
        loss = q_latent_loss + self.commitment_cost * e_latent_loss

        quantized = x + (quantized - x).detach()  # Straight-through estimator

        return quantized, loss, encoding_indices

class VQVAE(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_embeddings, embedding_dim, commitment_cost):
        super(VQVAE, self).__init__()
        self.encoder = Encoder(in_channels, hidden_channels, num_embeddings, embedding_dim)
        self.decoder = Decoder(embedding_dim, hidden_channels, in_channels)
        self.vq_layer = VectorQuantizer(num_embeddings, embedding_dim, commitment_cost)

    def forward(self, x):
        encoded = self.encoder(x)
        quantized, vq_loss, _ = self.vq_layer(encoded)
        decoded = self.decoder(quantized)
        return decoded, vq_loss

# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VQVAE(in_channels=3, hidden_channels=hidden_channels, num_embeddings=num_embeddings,
              embedding_dim=embedding_dim, commitment_cost=commitment_cost).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training Loop with Progress Bar
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # Use tqdm to display progress bar
    for images, _ in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
        images = images.to(device)

        # Forward pass
        reconstructed, vq_loss = model(images)
        recon_loss = criterion(reconstructed, images)
        loss = recon_loss + vq_loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Save the trained model
model_save_path = os.path.join(output_dir, "vqvae_model.pth")
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Epoch 1/10:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch [1/10], Loss: 0.1460


Epoch 2/10:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch [2/10], Loss: 0.1705


Epoch 3/10:   0%|          | 0/3 [00:00<?, ?it/s]

...

Epoch [8/10], Loss: 0.2742


Epoch 9/10:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch [9/10], Loss: 0.3540


Epoch 10/10:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch [10/10], Loss: 0.4635
Model saved to /Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Auto-Encoder/Trained Model/vqvae_model.pth
